In [6]:
import imageio
import ffmpeg

import numpy as np

import sys
from glob import glob

import site
site.addsitedir('../../ARGUS')
from ARGUSUtils import *

In [14]:
data_paths = sorted(glob("../../Data/TestingData/BAMC-PTX*Sliding/*.m??"))

In [21]:
def shape_ffmpeg( filename ):
    p = ffmpeg.probe(filename, select_streams='v');
    width = p['streams'][0]['width']
    height = p['streams'][0]['height']
    return height, width

def load_ffmpeg( filename, height, width ):
    p = ffmpeg.probe(filename, select_streams='v');
    width = p['streams'][0]['width']
    height = p['streams'][0]['height']

    in_bytes, _ = (
        ffmpeg
        .input(filename)
        .video # Video only (no audio).
        .output('pipe:', format='rawvideo', pix_fmt='gray')  # Set the output format to raw video in 8 bit grayscale
        .run(capture_stdout=True)
    )
    n_frames = len(in_bytes) // (height*width)  # Compute the number of frames.
    frames = np.frombuffer(in_bytes, np.uint8).reshape(n_frames, height, width) # Reshape buffer to array of n_frames frames (shape of each frame is (height, width)).
    return frames

In [22]:
def load_imageio( filename, height, width ):
    vid = imageio.get_reader(filename, 'ffmpeg')
    num_frame = vid.count_frames()
    img = np.empty((num_frame,height,width))
    count = 0
    for f in range(num_frame):
        img[count] = np.squeeze(np.mean(vid.get_data(f),axis=2))
        count += 1
    return img

In [23]:
height,width = shape_ffmpeg(data_paths[0])
num_vid = 1 #len(data_paths)

with time_this('load_ffmpeg'):
    for i in range(num_vid):
        img = load_ffmpeg(data_paths[i],height,width)

with time_this('load_imageio'):
    for i in range(num_vid):
        img = load_imageio(data_paths[i],height,width)

Time for load_ffmpeg is 1.3368921999999657
Time for load_imageio is 3.4808093999999983
